# Assignment II A
create a jupyter notebook to replace the add user interface for the assignment 2 of rt1. This interface will provide the user the capability of set and delete a terget.

## Inizialization
Import all the necessary modules

In [1]:
# Import necessary modules
import ipywidgets as widgets
import jupyros as jr
from IPython.display import display
import rospy
import actionlib
import assignment_2_2023
import assignment_2_2023.msg
from nav_msgs.msg import Odometry
from assignment_2_2023.msg import PlanningAction, PlanningActionGoal, RobotInfo


## User interface
In this part is possible to send a new target at the robot or delete the actual target. Is possible to write the new target inside the box and use the bottom below for send or delete the message.


In [3]:
rospy.init_node("jupyter_node")

interface = actionlib.SimpleActionClient("/reaching_goal", assignment_2_2023.msg.PlanningAction)
goal = assignment_2_2023.msg.PlanningGoal()

# Create text input fields for coordinates
x_input = widgets.FloatText(value=0.0, description='X Coordinate:')
y_input = widgets.FloatText(value=0.0, description='Y Coordinate:')

# Create a button to submit coordinates
submit_button = widgets.Button(description='Submit Coordinates')

# Create a button for delete the cordinate
delete_button = widgets.Button(description='Delete Coordinates')

# Display the widgets
vbox = widgets.VBox([x_input, y_input, submit_button, delete_button])
display(vbox)

#display(x_input, y_input, submit_button, delete_button)

# Define a function to handle button click
def on_submit_button_clicked(b):
    x = x_input.value
    y = y_input.value
    
    # Here you can add the code to send the coordinates to the robot
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y
    # Send goal to the action server
    interface.send_goal(goal)
    print(f'Coordinates submitted: X={x}, Y={y}')
    
def on_delete_button_clicked(b):
    if interface.get_state() == actionlib.GoalStatus.ACTIVE:
        interface.cancel_goal()
        print(f'Goal cancelled')
    else:
        print(f'There is no active goal to cancel')
# Attach the functions to the buttons
submit_button.on_click(on_submit_button_clicked)
delete_button.on_click(on_delete_button_clicked)




There is no active goal to cancel
There is no active goal to cancel
Coordinates submitted: X=5.0, Y=10.0
Goal cancelled
There is no active goal to cancel
There is no active goal to cancel
There is no active goal to cancel
There is no active goal to cancel
There is no active goal to cancel
There is no active goal to cancel
There is no active goal to cancel
There is no active goal to cancel
There is no active goal to cancel
There is no active goal to cancel
There is no active goal to cancel
There is no active goal to cancel
Coordinates submitted: X=5.0, Y=10.0
Coordinates submitted: X=5.0, Y=10.0
Coordinates submitted: X=5.0, Y=10.0
Coordinates submitted: X=5.0, Y=10.0
Coordinates submitted: X=5.0, Y=10.0
Coordinates submitted: X=5.0, Y=10.0
Coordinates submitted: X=5.0, Y=10.0
Coordinates submitted: X=5.0, Y=10.0
Coordinates submitted: X=5.0, Y=10.0


## Position of robot
In this section will be printed in a box text the position of the robot. For doing that I decide to subscribe to a topic /odom and print the value of the position of the robot. Those values are inside the mesage of type: msg.pose.pose.position and respectively on position x, y and z cordinates.

In [4]:
# Create FloatText widgets for displaying robot position
x_position_display = widgets.FloatText(description='X:')
y_position_display = widgets.FloatText(description='Y:')
z_position_display = widgets.FloatText(description='Z:')

# Define a callback function for the subscriber
def callback(msg):
    # Get the position from the message
    position = msg.pose.pose.position
    
    # Update the widget values
    x_position_display.value = position.x
    y_position_display.value = position.y
    z_position_display.value = position.z
    
# Create a subscriber
sub = rospy.Subscriber('/odom', Odometry, callback)

# Display the widgets
display(x_position_display, y_position_display, z_position_display)

FloatText(value=0.0, description='X:')

FloatText(value=0.0, description='Y:')

FloatText(value=0.0, description='Z:')

In [5]:
# Create FloatText widgets for displaying the last goal
x_goal_display = widgets.FloatText(description='Goal X:')
y_goal_display = widgets.FloatText(description='Goal Y:')

def callback(msg):
    # Get the last goal position from the message
    last_goal_position = msg.goal.target_pose.pose.position

    # Update the widget values
    x_goal_display.value = last_goal_position.x
    y_goal_display.value = last_goal_position.y

# Display the widgets
display(x_goal_display, y_goal_display)

FloatText(value=0.0, description='Goal X:')

FloatText(value=0.0, description='Goal Y:')